In [10]:
import os

# Get the directory of the current script
base_dir = os.getcwd()
try:
    if folder:
        base_dir = os.path.join(base_dir, folder)
except:
    pass

In [11]:
from rdflib import *

In [12]:
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

# Load Schema + Metadata

In [13]:
# Load your RDF ontology file
schema_metadata = Graph()
schema_metadata.parse(os.path.join(base_dir, '../Registration/Source_Graph_DP2.ttl'))


In [14]:
print(schema_metadata.serialize(format='turtle'))

## Data Contract

In [15]:
contract = schema_metadata
contract.bind('dqv', "http://www.w3.org/ns/dqv#")

Begin manual contract by adding the schema_metadata to the sdm

In [16]:
from rdflib import *
from hashlib import sha256
dataset_name = 'dicomimage'

### Create Contract

In [17]:
contract.add((abox[f'dc_{dataset_name}'],RDF.type, tbox.DataContract))
contract.add((abox[dataset_name],tbox.hasDC, abox[f'dc_{dataset_name}']))

### Add Mappings

In [18]:
contract.add((abox.map2, RDF.type, tbox.SchemaMapping))
contract.add((abox[f'dc_{dataset_name}'],tbox.hasMapping, abox.map2))

contract.add((abox.map2, tbox.mfrom, abox.PatientID))
contract.add((abox.map2, tbox.mto, abox.Subject))


In [19]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
contract.serialize(destination=os.path.join(base_dir, 'SDP2_C.ttl'), format=rdf_format)

### Add Policy

In [20]:
contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p1))
#contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p2))
#contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p3))

In [21]:
!pip install rdflib_jsonld
import json

In [22]:
def add_jsonld_instances(graph, path):
    # Adds JSON-LD instances to the graph
    with open(path, 'r') as f:
        json_ld_data = json.loads(f.read())
        instances = Graph().parse(data=json_ld_data, format='json-ld')
        graph += instances
    
    return graph

pc1

In [23]:
p1 = os.path.join(base_dir, '../../FederatedComputationalGovernance/ComputationalCatalogues/p1.json')
g = add_jsonld_instances(contract, p1)

pc2

In [24]:
#p2 = '/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/ComputationalCatalogues/p2.json'
#g = add_jsonld_instances(contract, p2)

p3

In [25]:
#p3 = '/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/ComputationalCatalogues/p3.json'
#g = add_jsonld_instances(contract, p3)

In [26]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
contract.serialize(destination=os.path.join(base_dir, 'SDP2_C.ttl'), format=rdf_format)

## SEND CONTRACT TO SDM

In [27]:
sdm = Graph().parse(os.path.join(base_dir, '../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'))

In [28]:
sdm += contract

In [29]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
sdm.serialize(destination=os.path.join(base_dir, '../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'), format=rdf_format)